In [1]:
# !pip install segmentation_models_pytorch

In [2]:
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
from mydataset2 import MyDataset
from tqdm import tqdm
from torch.cuda.amp import autocast
import segmentation_models_pytorch as smp
from data_augmentation import RandomRotation, RandomVerticalFlip, RandomHorizontalFlip, Compose, ToTensor

D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [3]:
batch_size = 1

In [4]:
train_set = MyDataset(root="data", is_train=True, transform=Compose([
    ToTensor(),
    RandomHorizontalFlip(),
    RandomVerticalFlip(),
    RandomRotation([0, 90, 180, 270]),
]), normalize=transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)))

val_set = MyDataset(root="data", is_train=False, transform=Compose([ToTensor()]),
                    normalize=transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)))

train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0
)

val_loader = DataLoader(
    val_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0
)
len(train_set), len(val_set), len(train_loader), len(val_loader)

(3800, 200, 3800, 200)

In [5]:
model0 = smp.FPN(
    in_channels=6,
    classes=2,
)
model0.cuda()
criterion0 = nn.CrossEntropyLoss()
optim0 = torch.optim.AdamW(model0.parameters(), lr=0.0001, weight_decay=1e-9)
scaler0 = torch.cuda.amp.GradScaler(enabled=True)

In [6]:
model1 = smp.DeepLabV3Plus(
    in_channels=6,
    classes=2,
)
model1.cuda()
criterion1 = nn.CrossEntropyLoss()
optim1 = torch.optim.AdamW(model1.parameters(), lr=0.0001, weight_decay=1e-9)
scaler1 = torch.cuda.amp.GradScaler(enabled=True)

In [7]:
model2 = smp.UnetPlusPlus(
    in_channels=6,
    classes=2,
)
model2.cuda()
criterion2 = nn.CrossEntropyLoss()
optim2 = torch.optim.AdamW(model2.parameters(), lr=0.0001, weight_decay=1e-9)
scaler2 = torch.cuda.amp.GradScaler(enabled=True)

In [8]:
models = [model0, model1, model2]
criterions = [criterion0, criterion1, criterion2]
scalers = [scaler0, scaler1, scaler2]
optims = [optim0, optim1, optim2]

In [9]:
def train_model(epoch):
    for model in models:
        model.train()
    print(f"Epoch {epoch} Training")
    with tqdm(train_loader, desc=str(epoch)) as it:
        for idx, (img, mask) in enumerate(it, 0):
            img, mask = img.cuda(), mask.cuda()
            mask = mask.long()
            mask = mask.squeeze(1)
            vote = []
            for model, optim, criterion, scaler in zip(models, optims, criterions, scalers):
                optim.zero_grad()
                with autocast():
                    outputs = model(img)
                    loss = criterion(outputs, mask)
                scaler.scale(loss).backward()
                scaler.step(optim)
                scaler.update()
                _, pred = torch.max(outputs.data, 1)
                vote.append(pred)
            vote = torch.stack(vote, dim=0)
            pred = torch.div(torch.sum(vote, dim=0), 2, rounding_mode="trunc")
            p, r, f1, iou = get_index(pred, mask)
            it.set_postfix_str(f"loss: {loss.item(): .4f} p: {p: .4f}  r: {r: .4f}  f1: {f1: .4f}  iou: {iou: .4f}")

In [10]:
def get_index(pred, label):
    eps = 1e-7
    tp = torch.sum(label * pred)
    fp = torch.sum(pred) - tp
    fn = torch.sum(label) - tp

    p = (tp + eps) / (tp + fp + eps)
    r = (tp + eps) / (tp + fn + eps)
    f1 = (2 * p * r + eps) / (p + r + eps)
    iou = (tp + eps) / (tp + fn + fp + eps)
    return p, r, f1, iou


def test_model(epoch):
    for model in models:
        model.eval()
    global max_score
    f1s = 0
    print(f"Epoch {epoch} Testing")
    with torch.no_grad():
        with tqdm(val_loader, desc=str(epoch)) as it:
            for img, mask in it:
                img, mask = img.cuda(), mask.cuda()
                mask = mask.squeeze(1)
                vote = []
                for model in models:
                    outputs = model(img)
                    _, pred = torch.max(outputs.data, 1)
                    vote.append(pred)
                vote = torch.stack(vote, dim=0)
                pred = torch.div(torch.sum(vote, dim=0), 2, rounding_mode="trunc")
                p, r, f1, iou = get_index(pred, mask)
                f1s += f1
                it.set_postfix_str(f"p: {p: .4f}  r: {r: .4f}  f1: {f1: .4f}  iou: {iou: .4f}")
    f1s /= len(val_loader)
    print("f1", f1s.item())
    if max_score < f1s:
        max_score = f1s
        print('max_score', max_score.item())

In [11]:
num_epoch = 10
max_score = 0
for epoch in range(0, num_epoch):
    train_model(epoch=epoch)
    test_model(epoch=epoch)
print("completed!")
print('max_score', max_score.item())

Epoch 0 Training


0:   0%|          | 0/3800 [00:02<?, ?it/s]


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 256, 1, 1])